In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, current_timestamp, to_timestamp, concat
from config_file import raw_path, processed_path
from common_func import add_timestamp

In [2]:
spark = SparkSession.builder.appName('ETL').getOrCreate()
spark

## Pit Stops Data Ingestion

In [3]:
pit_stop_df = spark.read.json(f"{raw_path}\pit_stops.json",multiLine=True)
pit_stop_df.show()

+--------+--------+---+------------+------+----+--------+
|driverId|duration|lap|milliseconds|raceId|stop|    time|
+--------+--------+---+------------+------+----+--------+
|     153|  26.898|  1|       26898|   841|   1|17:05:23|
|      30|  25.021|  1|       25021|   841|   1|17:05:52|
|      17|  23.426| 11|       23426|   841|   1|17:20:48|
|       4|  23.251| 12|       23251|   841|   1|17:22:34|
|      13|  23.842| 13|       23842|   841|   1|17:24:10|
|      22|  23.643| 13|       23643|   841|   1|17:24:29|
|      20|  22.603| 14|       22603|   841|   1|17:25:17|
|     814|  24.863| 14|       24863|   841|   1|17:26:03|
|     816|  25.259| 14|       25259|   841|   1|17:26:50|
|      67|  25.342| 15|       25342|   841|   1|17:27:34|
|       2|  22.994| 15|       22994|   841|   1|17:27:41|
|       1|  23.227| 16|       23227|   841|   1|17:28:24|
|     808|  24.535| 16|       24535|   841|   1|17:28:39|
|       3|  23.716| 16|       23716|   841|   1|17:29:00|
|     155|  24

In [4]:
pit_stop_df.printSchema()

root
 |-- driverId: long (nullable = true)
 |-- duration: string (nullable = true)
 |-- lap: long (nullable = true)
 |-- milliseconds: long (nullable = true)
 |-- raceId: long (nullable = true)
 |-- stop: long (nullable = true)
 |-- time: string (nullable = true)



In [5]:
pit_stop_df = pit_stop_df.withColumn('duration',pit_stop_df.duration.cast('float'))\
                        .withColumnRenamed('driverId','driver_Id')\
                        .withColumnRenamed('raceId','race_Id')

In [6]:
pit_stop_df = add_timestamp(pit_stop_df)

In [7]:
pit_stop_df.write.mode('overwrite').parquet(f"{processed_path}\Pit_Stop")